In [53]:
import sys
print(sys.executable)

/Users/drishtimannan/anaconda3/envs/bloby/bin/python


In [54]:
import pandas as pd
import tifffile
import random
import numpy as np
import matplotlib.pyplot as plt

 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

### Read the csv of the ground truth as a pandas dataframe. Here we are extracting centroid locations for all timepoints. This will be used to train and test LDA performance

In [49]:
#upload ground truth labels for generating training data for LDA
ground1=pd.read_csv('../dmannan1/LDA2/Substacks_with_SpineROIs/a1_tp1.csv')
ground2 = pd.read_csv('../dmannan1/LDA2/Substacks_with_SpineROIs/a2_tp2.csv')
ground3 = pd.read_csv("../dmannan1/LDA2/Substacks_with_SpineROIs/a3_tp3.csv")
ground4 = pd.read_csv('../dmannan1/LDA2/Substacks_with_SpineROIs/a4_tp4.csv')

print(len(ground1))
print(len(ground2))
print(len(ground3))
print(len(ground4))


222
195
218
207


In [50]:
### Change the panadas dataframe to numpy array and arrange the columns as Z,Y,X

ground_truth1 = np.asarray(ground1[['Z', 'Y', 'X']])
ground_truth2 = np.asarray(ground2[['Z', 'Y', 'X']])
ground_truth3 = np.asarray(ground3[['Z', 'Y', 'X']])
ground_truth4 = np.asarray(ground4[['Z', 'Y', 'X']])



### The csv given, the z-slices range from 1-16, but need to range from 0-15.

In [51]:
ground_truth1[:,0] = ground_truth1[:,0] - 1 ## -1 to make sure the dummy array index and ground_truth match
ground_truth2[:,0] = ground_truth2[:,0] - 1
ground_truth3[:,0] = ground_truth3[:,0] - 1
ground_truth4[:,0] = ground_truth4[:,0] - 1

# print(ground_truth1)
# print(ground_truth2)
# print(ground_truth3)
# print(ground_truth4)
len(ground_truth1)

[[  2  50   6]
 [  1   9  39]
 [  2   6  30]
 [  1  27  28]
 [  1  28  37]
 [  1  43  22]
 [  1  65  24]
 [  1   0  70]
 [  1  99   5]
 [  1  99  35]
 [  1  63  35]
 [  1  37  48]
 [  2 155  15]
 [  1 178  -1]
 [  1 167  12]
 [  1 200  17]
 [  2 205  38]
 [  1 220  56]
 [  1 232  40]
 [  1 232  50]
 [  1 254  10]
 [  2 279   9]
 [  1 305   1]
 [  1 292  44]
 [  2 340  26]
 [  1 280   1]
 [  2 326  29]
 [  1 317   0]
 [  2 330   4]
 [  2 328  70]
 [  1 310  65]
 [  2 348  12]
 [  2 246  58]
 [  2 265  72]
 [  1 247  68]
 [  1 237  59]
 [  1 252  76]
 [  2 226  86]
 [  2 229  99]
 [  2 152  45]
 [  1 150  55]
 [  1  69  84]
 [  1   0 103]
 [  1  29 121]
 [  1  37 115]
 [  1  67 122]
 [  1 163  94]
 [  1 173 105]
 [  2 175 115]
 [  1 185 100]
 [  2 196 109]
 [  2 196 136]
 [  2 206 134]
 [  1 190  51]
 [  2 203  56]
 [  1 351  26]
 [  1 310 108]
 [  1 259 114]
 [  1 270  52]
 [  2 133  88]
 [  1   4   3]
 [  1  42   0]
 [  1 145 138]
 [  1 164 155]
 [  2 197  84]
 [  2 222 121]
 [  2 226 

222

### We make sure that synapse cubes do not overlap with non-synapses in training and testing set
Extract centroid locations of known synapses and mark an area around the synapses 

In [52]:
dummy_array = np.zeros((16, 359, 359))##to keep track of the centroids and the area around it to make sure training 
#set doesn't have any overlapping areas. We need 359x359 by 16 arrays. In here provide the dimensions of tiff file 
# you are working with in z,y,x format
dummy_array.shape


(16, 359, 359)

### Finally need to extract non-synapses that don't overlap with synapse locations or their cubes

In [ ]:
def centroid_cubes(size_cube, array, centroid_locations): ##need to make cubes around our synapses to ensure they don't overlap with
##non-synapses

    sin=0
    for z, y ,x in zip(centroid_locations[:,0], centroid_locations[:,1], centroid_locations[:,2]):
        if x-size_cube<0 or x+size_cube>358 or y-size_cube<0 or y+size_cube>358:
            continue
        for l in range(y-size_cube,y+size_cube): ## 4.84 microns cubed cube is formed by a 22x22 rectangle around the centroid locations
            for m in range(x-size_cube,x+size_cube):
                array[z][l][m] = 1
        sin = sin+1


    return  array, sin


In [44]:
aa, ss =centroid_cubes(11,dummy_test,test_g1)
type(ss)
type(aa)

numpy.ndarray

In [45]:
def finding_non_synapses(size_cube, sin, array2):        
    num = 0 
    z_loc = []
    y_loc = []
    x_loc = []
    
    while num < sin:
    
    
        x_ns = np.random.randint(size_cube,200-size_cube)##discarding points within 5 pixels from boundary of 359x359x16 
    ##pixels because want to generate a 4.84 cube for mean intensity. Also, x and y values range from 0-358.
        y_ns = np.random.randint(size_cube,200-size_cube)
        z_ns = np.random.randint(0,16)
        if array2[z_ns][y_ns][x_ns] ==1 or array2[z_ns][y_ns-size_cube][x_ns-size_cube]==1 or array2[z_ns][y_ns+size_cube][x_ns+size_cube]==1 or array2[z_ns][y_ns-size_cube][x_ns+size_cube]==1 or array2[z_ns][y_ns+size_cube][x_ns-size_cube]==1:
            continue
        z_loc.append(z_ns)
        y_loc.append(y_ns)
        x_loc.append(x_ns)
        for s in range(y_ns-size_cube,y_ns+size_cube): ## 4.84 microns cubed cube is formed by a 10x10 rectangle around the centroid locations
            for p in range(x_ns-size_cube,x_ns+size_cube):
                 array2[z_ns][s][p] = 1
        num += 1
        print(num)
        

            
            
        
    
            
    
    return np.asarray((z_loc,y_loc,x_loc)).T
  
    

#updated_dummy = finding_non_syn(5, ground_truth1, dummy_array) ##this is the new non_syn training set

#updated_dummy

In [46]:
non_syn = finding_non_synapses(11,345,aa)
print(non_syn)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


### Read the image tiff stack 

In [33]:
#im1 = tifffile.imread('Substacks_with_SpineROIs/R04_tp1_substack (17-32).tif')
im_test = tifffile.imread('../dmannan1/LDA2/lids_smallest_stack.tif')

### Finally finding the mean intensity of the cubes around the centroids generated

In [ ]:
def generating_intensity(samples, cube_size, image):
    all_intensity = []
    b = 0


    for z, y, x in zip(samples[:,0], samples[:,1], samples[:,2]):##take the z,x,y locations to iterate over
        intensity = []
        if x-cube_size<0 or x+cube_size>199 or y-cube_size<0 or y+cube_size>199:
            continue



        for l in range(y-cube_size,y+cube_size): ## 1 microns cubed cube is formed by a 22x22 rectangle around the centroid locations
            for m in range(x-cube_size,x+cube_size):
                intensity.append(image[z,l,m])

        mean_intensity = np.mean(intensity)
        all_intensity.append(mean_intensity)
        b = b+1
            #print(all_intensity)



    return all_intensity, b

In [34]:
# def generating_intensity(samples, cube_size, image):
#     all_intensity = []
#     try:
        
#         for z, y, x in zip(samples[:,0], samples[:,1], samples[:,2]):##take the z,x,y locations to iterate over
#             intensity = []
            
#             for l in range(y-cube_size,y+cube_size): ## 1 microns cubed cube is formed by a 22x22 rectangle around the centroid locations
#                 for m in range(x-cube_size,x+cube_size):
#                     intensity.append(image[z,l,m])
            
#             mean_intensity = np.mean(intensity)
#             all_intensity.append(mean_intensity)
#             print(all_intensity)
#     except:
#         pass
#     return all_intensity


### Generating intensities for synapses and non-synapses

In [ ]:
s, a =centroid_cubes(11,test_g1)
# u = finding_non_synapses(11,s,a)
# synt = generating_intensity(test_g1,11,im_test)
# nst = generating_intensity(u,11,im_test)

s

In [ ]:
syn_in= generating_intensity(ground_truth1, 5)

syn_intensity = np.array(syn_in).reshape((222,1)) ## reshape the list to be a 2D array
syn_intensity.shape

In [ ]:
non_syn_in= generating_intensity(updated_dummy, 5)
non_syn_intensity = np.array(non_syn_in).reshape((222,1))
non_syn_intensity.shape

### Splitting the intensity arrays into training and testing:
Decide how much the intensity set will be split into testing and training set

In [ ]:
##training set, need the 1st 74 rows choosing 33% of the data to be training set
train_syn = syn_intensity[:111]
train_non_syn = non_syn_intensity[:111]
    

    
##Testing set, take next rows 
test_syn= syn_intensity[111:]
test_non_syn = non_syn_intensity[111:]

    
    
### The final training set:
train_data = np.vstack((train_syn, train_non_syn))
test_data = np.vstack((test_syn, test_non_syn))
### The final label generation, synapses belong to class 1, non-synpases belong to class 0:
train_labels = np.append(np.ones((len(train_syn))), np.zeros(len(train_non_syn)))
    
    
 

In [ ]:
print(train_syn.shape)
print(test_syn.shape)
print(train_labels.shape)
print(test_data.shape)

### Running LDA

In [ ]:
b = []
p = []
r = []
f = []
prediction_int_overlays = []
for i in np.arange(0.0, 1.0, 0.1):
    
    lda = LDA().fit(train_data, train_labels)
    probs_positive_class = lda.predict_proba(test_data)[:, 1]

    prediction = probs_positive_class > i
    prediction_int = prediction.astype(int)

    
    
    
       
    wrong_syn = 148-sum(prediction_int[:148]) ##know that the 1st 220 are synapses
    wrong_non_syn=sum(prediction_int[148:]) ###know that the last 220 are non-synapses
    precision = sum(prediction_int[:148])/ (sum(prediction_int[:220])+wrong_non_syn)
    recall = sum(prediction_int[:148])/ (sum(prediction_int[:148])+(148-sum(prediction_int[:148])))
    if precision and recall != 0:
        f1 = (2*precision*recall)/(precision + recall)
    else:
        f1 = 0 
    print(prediction_int, "Total wrong synapses:", wrong_syn, "Total wrong non-synapses:", wrong_non_syn, "Precision:",
         precision, "Recall:", recall, "F1", f1)
    p.append(precision)
    r.append(recall)
    b.append(i)
    f.append(f1)
    prediction_int_overlays.append(prediction_int)
    
plt.plot(b, p)
plt.plot(b, r)
plt.plot(b, f)
plt.xlabel('Decision boundary threshold')
plt.ylabel('Value of the labelled metric')

plt.legend(['Precision', 'Recall', "F1"], loc='upper right')
plt.title('Mean intensity of a 4.84 ${\mu}m^3$ cube and LDA performance by varying decision threshold')
plt.show()

    
    



In [ ]:
## Running it on timpeoint 1

lda = LDA().fit(train_data, train_labels)
probs_positive_class = lda.predict_proba(test_data)[:,1]

prediction = probs_positive_class > 0.
prediction_int = prediction.astype(int)
print(prediction_int)

In [ ]:
clf = LDA()
clf.fit(train_data, train_labels)
#predict_proba(test_data)
j = clf.predict(test_data)
print(j)
print(clf.predict_proba(test_data))
print(clf.predict_proba(test_data)[:,1])


In [ ]:
probs_positive_class.shape

In [ ]:
sum(j[148:]) 